## Validation of ipcoal sequence simulator

In this notebook we implement ipcoal simulations that use either `seqgen` or our `seqmodel` as the underlying sequence algorithm for mutating sequences evolving on trees. We show that under all models that we currently support in the `ipcoal` implementation matches the outputs of `seqgen`. 

In [2]:
import ipcoal
import toytree
import numpy as np
import pandas as pd
import toyplot
from concurrent.futures import ProcessPoolExecutor

### A species tree
We will evolve sequences on genealogies that are sampled from a species tree. An example species tree is shown below generated from `toytree`. 


In [3]:
# generate a random species tree topology
tree = toytree.rtree.unittree(ntips=8, treeheight=1e6, seed=123)

# draw the species tree
canvas, axes = tree.draw(ts='p');

# add a title
canvas.text(
    x=canvas.width / 2., 
    y=20,
    text="Species tree", 
    style={"font-size": "14px"},
);

<svg class="toyplot-canvas-Canvas" height="260.0px" id="tc9fbe5ed19f64c4987d40858d4f1bea4" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 350.0 260.0" width="350.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> r6 r4 r7 r2 r5 r0 r3 r1 idx: 0
name: r6
dist: 500000
support: 100
height: 0 0 idx: 1
name: r4
dist: 500000
support: 100
height: 0 1 idx: 2
name: r7
dist: 500000
support: 100
height: 0 2 idx: 3
name: r2
dist: 500000
support: 100
height: 0 3 idx: 4
name: r5
dist: 250000
support: 100
height: 0 4 idx: 5
name: r0
dist: 250000
support: 100
height: 0 5 idx: 6
name: r3
dist: 500000
support: 100
height: 0 6 idx: 7
name: r1
dist: 750000
support: 100
height: 0 7 idx: 8
name: 8
dist: 250000
support: 100
height: 250000 8 idx: 9
name: 9
dist: 250000
support: 100
height: 500000 9 idx: 10
name: 10
dist: 250000
support: 100
height: 500000 10 idx: 11
name: 11
dist: 250000
support: 100
height: 500000 11 idx: 12
name: 12
dist: 250000
support: 100
height: 750000 12 idx: 13
name: 13
dist: 250000
support: 100
height: 750000 13 idx: 14
name: 14
dist: 250000
support: 100
height: 1000000 14 0 500000 1000000 Species tree

### Define a demographic model based on the species tree
Using `ipcoal` we can sample a single genealogy under a demographic model defined by the divergence times in the species tree above, and with an effective population size parameter applied to all edges of the tree. 

In [4]:
# define an ipcoal model, simulate trees and show result table
mod = ipcoal.Model(tree=tree, Ne=1e5, seed=1234)
mod.sim_trees(1)
mod.df.head()

,locus,start,end,nbps,nsnps,genealogy
0,0,0,1,1,0,"((r1:859988,(r3:666450,(..."


### Draw one simulated genealogy

Here I use the `fixed_order=...` argument to toytree so that it will plot the tips in the same order as in the species tree above. This makes it easier to see the differences between the two trees. 

In [5]:
# load the resulting genealogy as a toytree
genealogy = toytree.tree(mod.df.genealogy[0], fixed_order=tree.get_tip_labels())

# draw the tree
canvas, axes = genealogy.draw(ts='c', tip_labels=True);

# add a title
canvas.text(
    x=canvas.width / 2., 
    y=20,
    text="Gene tree", 
    style={"font-size": "14px"},
);

<svg class="toyplot-canvas-Canvas" height="260.0px" id="t16820ae777764eb188b98fbd0c8c23f5" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 350.0 260.0" width="350.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> r6 r4 r7 r2 r5 r0 r3 r1 0 721508 1443016 Gene tree

### Simulate sequence data
In addition to the `sim_trees()` function call used above, which only samples genealogies evolving under the defined model, `ipcoal` can  also simulate SNPs or loci evolving on genealogies by using the function calls `.sim_snps()` or `.sim_loci()`. In this case a markov model of molecular substitutions will be applied to mutate sites along the edges of the tree. The default option is to evolve sites under the Jukes-Cantor model, but you can provide additional parameter options to implement more complex models similar to the `seqgen` program. 

In [5]:
# init the model
mod = ipcoal.Model(
    tree, 
    Ne=1e5,
    mut=1e-8,
    recomb=0,
    substitution_model={
        "state_frequencies": (0.25, 0.25, 0.25, 0.25),
        "kappa": 1.0,
    },
    seed=123,
)

You can view a summary of the substitution model after initializing the `ipcoal` model object to see the effect of substitution model parameters on the instantaneous rate matrix. 

In [6]:
mod.get_substitution_model_summary()

state_frequencies:
    A     C     G     T
 0.25  0.25  0.25  0.25

kappa: 1.0
ts/tv: 0.5

instantaneous transition rate matrix:
        A       C       G       T
A -1.0000  0.3333  0.3333  0.3333
C  0.3333 -1.0000  0.3333  0.3333
G  0.3333  0.3333 -1.0000  0.3333
T  0.3333  0.3333  0.3333 -1.0000


### Evolve sequences in ipcoal using the `SeqModel` class

First we will generate data using the pure Python implementation in `ipcoal` which we call SeqModel. Then we will compare our results with data generated under the same parameter settings in `seqgen`. There is of course a lot of stochasticity in the evolutionary process, so to validate the two classes are returning similar results we will simulate data on a single genealogy (nloci=1) and for many sites (nsites=1e6). 

In [7]:
# simulate one locus
mod.sim_loci(nloci=1, nsites=1e6)

# calculate genetic distances
seqmodel_dists = mod.get_pairwise_distances()
seqmodel_dists

,r0,r1,r2,r3,r4,r5,r6,r7
r0,0.000000,0.026063,0.016022,0.025869,0.026036,0.025868,0.025914,0.025798
r1,0.026063,0.000000,0.026113,0.022914,0.010437,0.022932,0.022944,0.022879
r2,0.016022,0.026113,0.000000,0.025983,0.026111,0.025933,0.026006,0.025898
r3,0.025869,0.022914,0.025983,0.000000,0.022914,0.015847,0.011769,0.015802
r4,0.026036,0.010437,0.026111,0.022914,0.000000,0.022908,0.022943,0.022857
r5,0.025868,0.022932,0.025933,0.015847,0.022908,0.000000,0.015874,0.012134
r6,0.025914,0.022944,0.026006,0.011769,0.022943,0.015874,0.000000,0.015860
r7,0.025798,0.022879,0.025898,0.015802,0.022857,0.012134,0.015860,0.000000


### Evolve sequences in ipcoal using the `SeqGen` class
Here we implement the same model but use a subprocess call to pass the  genealogy and substution model arguments to the `seqgen` binary to perform the sequence simulation. 


In [8]:
# re-init the model (we want to start from the same seed)
mod = ipcoal.Model(
    tree, 
    Ne=1e5,
    mut=1e-8,
    recomb=0,
    substitution_model={
        "state_frequencies": (0.25, 0.25, 0.25, 0.25),
        "kappa": 1.0,
    },
    seed=123,
)

# simulate one locus this time using seqgen
mod.sim_loci(nloci=1, nsites=1e6, seqgen=True)

# calculate genetic distances
seqgen_dists = mod.get_pairwise_distances()
seqgen_dists

,r0,r1,r2,r3,r4,r5,r6,r7
r0,0.000000,0.026128,0.015953,0.026125,0.025972,0.026130,0.026092,0.025935
r1,0.026128,0.000000,0.026053,0.022988,0.010390,0.023034,0.022971,0.022825
r2,0.015953,0.026053,0.000000,0.026024,0.025885,0.026040,0.025972,0.025838
r3,0.026125,0.022988,0.026024,0.000000,0.022857,0.016070,0.011739,0.015832
r4,0.025972,0.010390,0.025885,0.022857,0.000000,0.022872,0.022824,0.022677
r5,0.026130,0.023034,0.026040,0.016070,0.022872,0.000000,0.016040,0.012187
r6,0.026092,0.022971,0.025972,0.011739,0.022824,0.016040,0.000000,0.015810
r7,0.025935,0.022825,0.025838,0.015832,0.022677,0.012187,0.015810,0.000000


### Are the results close enough within random expectations?

In [14]:
# are the values close to within a high tolerance?
np.allclose(seqmodel_dists, seqgen_dists, rtol=1e-1)

True

In [15]:
toyplot.matrix(
    seqgen_dists - seqmodel_dists,
    width=400, height=400,
    margin=10,
);

<svg class="toyplot-canvas-Canvas" height="400.0px" id="t24fd588df32e42e4a0095d4e3559caba" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 400.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> 0 1 2 3 4 5 6 7 0 0.000000 0.000065 -0.000069 0.000256 -0.000064 0.000262 0.000178 0.000137 1 0.000065 0.000000 -0.000060 0.000074 -0.000047 0.000102 0.000027 -0.000054 2 -0.000069 -0.000060 0.000000 0.000041 -0.000226 0.000107 -0.000034 -0.000060 3 0.000256 0.000074 0.000041 0.000000 -0.000057 0.000223 -0.000030 0.000030 4 -0.000064 -0.000047 -0.000226 -0.000057 0.000000 -0.000036 -0.000119 -0.000180 5 0.000262 0.000102 0.000107 0.000223 -0.000036 0.000000 0.000166 0.000053 6 0.000178 0.000027 -0.000034 -0.000030 -0.000119 0.000166 0.000000 -0.000050 7 0.000137 -0.000054 -0.000060 0.000030 -0.000180 0.000053 -0.000050 0.000000

### The results get more similar as more data is simulated?

In [7]:
def func(tree, seed, rep, nsites, kwargs):
    # re-init the model (we want to start from the same seed)
    mod1 = ipcoal.Model(
        tree, 
        Ne=1e5,
        mut=1e-8,
        recomb=0,
        substitution_model=kwargs,
        seed=seed,
    )

    # simulate one locus
    mod1.sim_loci(nloci=1, nsites=nsites)

    # calculate genetic distances
    seqmod_dists = mod1.get_pairwise_distances()

    # re-init the model (we want to start from the same seed)
    mod2 = ipcoal.Model(
        tree, 
        Ne=1e5,
        mut=1e-8,    
        recomb=0,
        substitution_model=kwargs,
        seed=seed,
    )
    # simulate one locus
    mod2.sim_loci(nloci=1, nsites=nsites, seqgen=True)

    # calculate genetic distances
    seqgen_dists = mod2.get_pairwise_distances()

    # get distance between matrices
    dist1 = (seqgen_dists - seqmod_dists).abs().sum().sum()
    dist2 = (seqgen_dists - seqmod_dists).values.flatten().mean()
    return nsites, rep, dist1, dist2

### Testing JC model distances

In [72]:
nreps = 16
nsites = [1e3, 5e3, 1e4, 5e4, 1e5, 5e5, 1e6, 5e6]
nvals = len(nsites)

# first test simple JC model distances
kwargs = {
    "state_frequencies": (0.25, 0.25, 0.25, 0.25),
    "kappa": 1.0,
}

with ProcessPoolExecutor(max_workers=8) as executor:
    # simulate many replicates of diff size to fill dataframe
    idx = 0
    results = {}
    for ns in nsites:
        for rep in range(nreps):

            # random seed for this rep
            seed = np.random.randint(0, 1e8)
            future = executor.submit(func, *(tree, seed, rep, ns, kwargs))
            results[idx] = future
            idx += 1

In [73]:
# store result in dataframe
data = pd.DataFrame({
    "nsites": np.zeros(nreps * nvals),
    "rep": np.zeros(nreps * nvals),
    "dist": np.zeros(nreps * nvals, dtype=float),
})

In [74]:
for idx in results:
    nsites, rep, dist1, dist2 = results[idx].result()
    data.nsites[idx] = nsites
    data.rep[idx] = rep
    data.dist[idx] = dist1

In [75]:
data.groupby("nsites").apply(np.mean)

,nsites,rep,dist
nsites,,,
1000.0,1000.0,7.5,0.276250
5000.0,5000.0,7.5,0.133625
10000.0,10000.0,7.5,0.095113
50000.0,50000.0,7.5,0.036420
100000.0,100000.0,7.5,0.023785
500000.0,500000.0,7.5,0.011747
1000000.0,1000000.0,7.5,0.010202
5000000.0,5000000.0,7.5,0.004104


In [76]:
# plot mean difference against data size
c, a, m = toyplot.scatterplot(
    data.nsites,
    data.dist, 
    height=300, 
    width=350, 
    size=10, 
    opacity=0.5,
    xscale='log',
    ylabel="sum of pairwise distances",
    xlabel="locus length",
);

# add line at zero
a.hlines(0, style={"stroke-width": 2, "stroke-dasharray": 5});
toyplot.pdf.render(c, "figures/seqgen_compare_loci_JC.pdf")

<svg class="toyplot-canvas-Canvas" height="300.0px" id="t9b16c6db3270490ba86fb04a98698d12" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 350.0 300.0" width="350.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> 10 3 10 4 10 5 10 6 10 7 locus length 0.0 0.1 0.2 0.3 0.4 mean pairwise distance

### The same is true with a more complex model (HKY)
Here the state frequencies and the ts/tv ratio are both set to non-default values to implement the HKY model. 

In [8]:
nreps = 16
nsites = [1e3, 5e3, 1e4, 5e4, 1e5, 5e5, 1e6]
nvals = len(nsites)

# first test simple JC model distances
kwargs = {
    "state_frequencies": (0.2, 0.1, 0.3, 0.4),
    "kappa": 0.5,
}

with ProcessPoolExecutor(max_workers=8) as executor:
    # simulate many replicates of diff size to fill dataframe
    idx = 0
    results = {}
    for ns in nsites:
        for rep in range(nreps):

            # random seed for this rep
            seed = np.random.randint(0, 1e8)
            future = executor.submit(func, *(tree, seed, rep, ns, kwargs))
            results[idx] = future
            idx += 1

In [9]:
# store result in dataframe
data = pd.DataFrame({
    "nsites": np.zeros(nreps * nvals),
    "rep": np.zeros(nreps * nvals),
    "dist": np.zeros(nreps * nvals, dtype=float),
})

In [10]:
for idx in results:
    nsites, rep, dist1, dist2 = results[idx].result()
    data.nsites[idx] = nsites
    data.rep[idx] = rep
    data.dist[idx] = dist1

In [11]:
# plot mean difference against data size
c, a, m = toyplot.scatterplot(
    data.nsites,
    data.dist, 
    height=300, 
    width=350, 
    size=10, 
    opacity=0.5,
    xscale='log',
    ylabel="sum of pairwise distances",
    xlabel="locus length",
);

# add line at zero
a.hlines(0, style={"stroke-width": 2, "stroke-dasharray": 5});
toyplot.pdf.render(c, "figures/seqgen_compare_loci_HKY.pdf")

<svg class="toyplot-canvas-Canvas" height="300.0px" id="t8da466792edc4b97bf6e57f0fc33e83d" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 350.0 300.0" width="350.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> 10 3 10 4 10 5 10 6 locus length 0.0 0.1 0.2 0.3 0.4 0.5 sum of pairwise distances

### Show the same is true when simulating unlinked SNPs

This will take longer than `.sim_loci()` to converge on the same answer because it is stochastic whether a SNP falls on an individual genealogy or not, and so the two methods will be simulating on the same *distribution* of genealogies, but not actually on the same exact genealogies most of the time. Still the genetic distances should converge when sampling over a very large distribution of genealogies. 

In [14]:
def func_snps(tree, seed, rep, nsites, kwargs):
    # re-init the model (we want to start from the same seed)
    mod1 = ipcoal.Model(
        tree, 
        Ne=1e5,
        mut=1e-8,
        recomb=0,
        substitution_model=kwargs,
        seed=seed,
    )

    # simulate one locus
    mod1.sim_snps(nsites)

    # calculate genetic distances
    seqmod_dists = mod1.get_pairwise_distances()

    # re-init the model (we want to start from the same seed)
    mod2 = ipcoal.Model(
        tree, 
        Ne=1e5,
        mut=1e-8,    
        recomb=0,
        substitution_model=kwargs,
        seed=seed,
    )
    # simulate one locus
    mod2.sim_snps(nsites, seqgen=True)

    # calculate genetic distances
    seqgen_dists = mod2.get_pairwise_distances()

    # get distance between matrices
    dist1 = (seqgen_dists - seqmod_dists).abs().sum().sum()
    dist2 = (seqgen_dists - seqmod_dists).values.flatten().mean()
    return nsites, rep, dist1, dist2

In [15]:
nreps = 16
nsites = [100, 500, 1000, 1500, 2000, 2500, 3000, 3500, 4000, 4500, 5000]
nvals = len(nsites)

# first test simple JC model distances
kwargs = {
    "state_frequencies": (0.2, 0.1, 0.3, 0.4),
    "kappa": 0.5,
}

with ProcessPoolExecutor(max_workers=8) as executor:
    # simulate many replicates of diff size to fill dataframe
    idx = 0
    results3 = {}
    for ns in nsites:
        for rep in range(nreps):

            # random seed for this rep
            seed = np.random.randint(0, 1e8)
            future = executor.submit(func_snps, *(tree, seed, rep, ns, kwargs))
            results3[idx] = future
            idx += 1

In [16]:
# store result in dataframe
data3 = pd.DataFrame({
    "nsites": np.zeros(nreps * nvals),
    "rep": np.zeros(nreps * nvals),
    "dist": np.zeros(nreps * nvals, dtype=float),
})

In [26]:
mod.sim_snps(10)

In [27]:
mod.get_pairwise_distances()

AxisError: axis 1 is out of bounds for array of dimension 1

In [18]:
for idx in results3:
    nsites, rep, dist1, dist2 = results3[idx].result()
    data3.nsites[idx] = nsites
    data3.rep[idx] = rep
    data3.dist[idx] = dist1

AxisError: axis 1 is out of bounds for array of dimension 1

In [ ]:
toyplot.pdf.render(c, "figures/seqgen_compare_snps_HKY.pdf")